# Detecção e classificação de objetos usando OpenCV
_Desenvolvido em Maio/2021 (repostado)_

Neste projeto, irei usar Visão Computacional com OpenCV, e Inteligência Artificial com Tensorflow e Keras, para fazer detecção e classificação de objetos em vídeo.

In [ ]:
#Primeiro, importamos os pacotes necessários
import cv2
import numpy as np
import tensorflow as tf

In [ ]:
#Depois, importamos o pacote pytube, que baixa vídeos direto do YouTube. Aqui, peguei um vídeo curto de poucos segundos de pessoas andando em uma rua.
!python -m pip install git+https://github.com/nficano/pytube
import pytube
path = 'https://www.youtube.com/watch?v=pk96gqasGBQ&ab_channel=TRIDE'
pytube.YouTube(path).streams.get_highest_resolution().download()

  Cloning https://github.com/nficano/pytube to /tmp/pip-req-build-_ntw5g7s
  Running command git clone -q https://github.com/nficano/pytube /tmp/pip-req-build-_ntw5g7s
  Created wheel for pytube: filename=pytube-10.7.2-cp37-none-any.whl size=42996 sha256=a10688fbcac29ca7429def254ef151c7b868e03b30a953ce67c71908c431c6b2
  Stored in directory: /tmp/pip-ephem-wheel-cache-_y9721qe/wheels/44/da/40/3b5e03abe33a91895343814fb44b309512375408f4a909555b
Successfully built pytube


'/content/Test video for Object Detection  TRIDE.mp4'

In [ ]:
#Em seguida, importamos o modelo de rede neural pré-treinada MobileNet, dentro do Keras API, com os pesos do dataset Imagenet
#Sobre o ImageNet: https://www.image-net.org/
#Sobre MobileNet: https://blog.fellyph.com.br/javascript/o-que-e-mobilenet/
model = tf.keras.applications.mobilenet.MobileNet(weights = 'imagenet', include_top = True)
model.summary()

#Note que a camada de inputs é do formato (224,224,3), logo toda região de interesse (ROI) detectada por frame precisa ser ajustada ao tamanho (224,224,3)
#O 3 em (224,224,3) indica que a imagem tem três camadas de cores (BGR - Blue-Green-Red)

17227776/17225924 [==============================] - 1s 0us/step
Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________

In [ ]:
import requests
imagenet_classes = requests.get(
    'https://gist.githubusercontent.com/yrevar/942d3a0ac09ec9e5eb3a/raw/238f720ff059c1f82f368259d1ca4ffa5dd8f9f5/imagenet1000_clsidx_to_labels.txt')
imagenet_classes = eval(imagenet_classes.text)

In [ ]:
#Detectando os objetos e armazenando os frames em um novo vídeo
cap = cv2.VideoCapture('Test video for Object Detection  TRIDE.mp4')
bg_substractor = cv2.createBackgroundSubtractorKNN() #Substrator de plano de fundo
bg_substractor.setHistory(20) #Lê 20 frames antes de começar as transformações
erode_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3)) #Núcleo de erosão 3x3
dilate_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (8,3)) #Núcleo de dilatação 8x3

fps = cap.get(cv2.CAP_PROP_FPS) #Frames por segundo do vídeo
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))) #Altura e largura de cada frame do vídeo
fourcc = cv2.VideoWriter_fourcc(*'I420') #'Gravador' de frames em um vídeo no formato AVI
out = cv2.VideoWriter('output.avi', fourcc, fps, size) 

success, frame = cap.read()
while success:
  fg_mask = bg_substractor.apply(frame) #foreground mask (máscara de primeiro plano)
  _, thresh = cv2.threshold(fg_mask, 127, 255, cv2.THRESH_BINARY) #delimitador, transforma qualquer pixel mais claro que o cinza em branco, e qualquer mais escuro em cinza em preto
      
  cv2.erode(thresh, erode_kernel, thresh, iterations = 2) #Erosão em foreground mask delimitada
  cv2.dilate(thresh, dilate_kernel, thresh, iterations = 2) #Dilatação em foreground mask delimitada
      
  contours, hier = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) #Detecção de contornos
  for c in contours:
    count = 0
    if cv2.contourArea(c) > 750:
      (x,y,w,h) = cv2.boundingRect(c) #Delimita um retângulo com o contorno dentro 
      cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2) #Desenha um retângulo ao redor do objeto detectado
      roi_frame = frame[y:y+h, x:x+w] / 255 #Região de interesse detectada no frame
      roi_frame = cv2.resize(roi_frame, (224,224)) #Redimensionando a região de interesse para o tamanho 224x224
      roi_label = imagenet_classes[np.argmax(model.predict(np.array([roi_frame], dtype = np.float32)))] #Classificação da região de interesse
      cv2.putText(frame, f'{roi_label}', (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 1) #Classe a qual pertence o objeto detectado
      print(f'Frame written.')

  out.write(frame) #Escreve o frame alterado

  success, frame = cap.read() #Lê um novo frame

In [ ]:
#Conversão do vídeo do formato avi para mp4
!ffmpeg -i output.avi output.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
#Download para a máquina, onde irei fazer upload dentro do repositório do GitHub
from google.colab import files 
files.download('output.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Este com certeza não é o jeito mais efetivo de se fazer. Por algum motivo, não consegui de jeito nenhum implementar o módulo cv2.dnn.readNet para usar a rede neural pré-treinada de modo eficiente.

Passei horas atrás de uma solução para poder usar este módulo sem ter esse "delay" (o vídeo de menos de 30s que fiz upload levou mais de 8min para que ficasse pronta a detecção e classificação do jeito que fiz, a famosa GAMBIARRA).

Porém, foi o jeito que encontrei no momento. Espero que implementem uma solução para esse problema neste módulo (tem a ver com congelar os grafos da rede pré-treinada do Tensorflow).